<a href="https://colab.research.google.com/github/thyagaraaj-oss/D.S.Thyagaraaj/blob/main/Kaggle_Capstone_Project_Ideas_%E2%80%93_Diet_%26_Nutrition_Assistant_Agents_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Agent Diet & Nutrition Project (Colab Notebook)

This notebook creates the project files using `%%writefile` magics, provides a runnable demo cell, and zips the project.

In [214]:
# Create project folders
!mkdir -p project/agents project/tools project/memory project/core
!ls -R project || true

project:
agents	core	       memory	    requirements.txt  tools
app.py	main_agent.py  __pycache__  run_demo.py

project/agents:
evaluator.py  planner.py  __pycache__  worker.py

project/agents/__pycache__:
evaluator.cpython-312.pyc  planner.cpython-312.pyc  worker.cpython-312.pyc

project/core:
a2a_protocol.py  context_engineering.py  observability.py  __pycache__

project/core/__pycache__:
context_engineering.cpython-312.pyc  observability.cpython-312.pyc

project/memory:
__pycache__  session_memory.py

project/memory/__pycache__:
session_memory.cpython-312.pyc

project/__pycache__:
main_agent.cpython-312.pyc

project/tools:
__pycache__  tools.py

project/tools/__pycache__:
tools.cpython-312.pyc


In [215]:
%%writefile project/agents/planner.py
from typing import Dict, Any

class Planner:
    def __init__(self, memory=None):
        self.memory = memory

    def plan(self, user_input: str) -> Dict[str, Any]:
        """
        Simple planner that interprets user input and creates a plan dict.
        """
        plan = {
            "intent": "generate_meal_plan" if "meal" in user_input.lower() or "diet" in user_input.lower() else "general",
            "user_input": user_input,
            "steps": ["parse_requirements", "generate_candidates", "finalize"]
        }
        return plan


Overwriting project/agents/planner.py


In [216]:
%%writefile project/agents/worker.py
from typing import Dict, Any, List
from project.tools.tools import NutritionDB

class Worker:
    def __init__(self, memory=None):
        self.memory = memory
        self.db = NutritionDB()

    def work(self, plan: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generates candidate meal items or responses based on the plan.
        """
        intent = plan.get("intent", "general")
        user_input = plan.get("user_input", "")
        if intent == "generate_meal_plan":
            # Very simple demo: return a single-day meal plan
            meals = [
                {"name": "Oatmeal with banana", "calories": 350, "protein_g": 10},
                {"name": "Grilled chicken salad", "calories": 450, "protein_g": 35},
                {"name": "Lentil soup with rice", "calories": 500, "protein_g": 20}
            ]
            return {"candidates": meals, "note": "Generated a basic 1-day plan"}
        else:
            # Fallback: use the DB to lookup a sample food
            item = self.db.lookup(user_input.split()[0]) if user_input else {"name": "water", "calories": 0}
            return {"candidates": [item], "note": "Lookup fallback"}


Overwriting project/agents/worker.py


In [217]:
%%writefile project/agents/evaluator.py
from typing import Dict, Any, List

class Evaluator:
    def __init__(self, memory=None):
        self.memory = memory

    def evaluate(self, candidates: List[Dict[str, Any]], constraints: Dict[str, Any]=None) -> Dict[str, Any]:
        """
        Very simple evaluator that scores candidates by closeness to calorie target if provided.
        """
        constraints = constraints or {}
        calorie_target = constraints.get("calories")
        scored = []
        for c in candidates:
            cal = c.get("calories", 0)
            score = 1.0
            if calorie_target:
                score = max(0.0, 1.0 - abs(cal - calorie_target) / max(1, calorie_target))
            scored.append({"item": c, "score": score})
        scored = sorted(scored, key=lambda x: x["score"], reverse=True)
        return {"scored": scored, "best": scored[0] if scored else None}


Overwriting project/agents/evaluator.py


In [218]:
%%writefile project/tools/tools.py
from typing import Dict, Any

class NutritionDB:
    """
    Mock nutrition database for demo purposes.
    """
    def __init__(self):
        self.data = {
            "oatmeal": {"name": "Oatmeal", "calories": 150, "protein_g": 5},
            "banana": {"name": "Banana", "calories": 100, "protein_g": 1},
            "chicken": {"name": "Grilled chicken", "calories": 250, "protein_g": 30},
            "lentils": {"name": "Lentil soup", "calories": 300, "protein_g": 18},
            "rice": {"name": "Rice", "calories": 200, "protein_g": 4},
            "water": {"name": "Water", "calories": 0, "protein_g": 0}
        }

    def lookup(self, key: str) -> Dict[str, Any]:
        key = key.lower()
        return self.data.get(key, {"name": key, "calories": 100, "protein_g": 5})


Overwriting project/tools/tools.py


In [219]:
%%writefile project/memory/session_memory.py
from typing import Dict, Any

class SessionMemory:
    def __init__(self):
        self.store: Dict[str, Any] = {}

    def get(self, key: str, default=None):
        return self.store.get(key, default)

    def set(self, key: str, value):
        self.store[key] = value

    def append_history(self, key: str, value):
        hist = self.store.get(key, [])
        hist.append(value)
        self.store[key] = hist


Overwriting project/memory/session_memory.py


In [220]:
%%writefile project/core/context_engineering.py
from typing import Dict, Any

def build_context(memory: Dict[str, Any], user_input: str) -> Dict[str, Any]:
    """
    Builds a context object for agents using memory and current input.
    """
    context = {
        "user_input": user_input,
        "user_profile": memory.get("profile", {})
    }
    return context


Overwriting project/core/context_engineering.py


In [221]:
%%writefile project/core/observability.py
import time
from typing import Dict, Any

class Observability:
    def __init__(self):
        self.logs = []

    def log(self, message: str, meta: Dict[str, Any]=None):
        entry = {"ts": time.time(), "message": message, "meta": meta or {}}
        self.logs.append(entry)

    def get_logs(self):
        return self.logs


Overwriting project/core/observability.py


In [222]:
%%writefile project/core/a2a_protocol.py
from typing import Dict, Any

def serialize_message(sender: str, payload: Dict[str, Any]) -> Dict[str, Any]:
    return {"sender": sender, "payload": payload}

def deserialize_message(message: Dict[str, Any]) -> Dict[str, Any]:
    return message.get("payload", {})


Overwriting project/core/a2a_protocol.py


In [223]:
%%writefile project/main_agent.py
from typing import Dict, Any
from project.memory.session_memory import SessionMemory
from project.agents.planner import Planner
from project.agents.worker import Worker
from project.agents.evaluator import Evaluator
from project.core.context_engineering import build_context
from project.core.observability import Observability

class MainAgent:
    def __init__(self):
        self.memory = SessionMemory()
        self.planner = Planner(memory=self.memory)
        self.worker = Worker(memory=self.memory)
        self.evaluator = Evaluator(memory=self.memory)
        self.obs = Observability()

    def handle_message(self, user_input: str) -> Dict[str, Any]:
        # Build context
        context = build_context(self.memory.store, user_input)
        self.obs.log("Received input", {"input": user_input})

        # Planner creates a plan
        plan = self.planner.plan(user_input)
        self.obs.log("Plan created", {"plan": plan})

        # Worker generates candidates
        work_result = self.worker.work(plan)
        candidates = work_result.get("candidates", [])
        self.obs.log("Worker produced candidates", {"count": len(candidates)})

        # Evaluator scores candidates
        constraints = {}
        # if user has target calories in memory, use it
        cal_target = self.memory.get("calorie_target")
        if cal_target:
            constraints["calories"] = cal_target
        eval_result = self.evaluator.evaluate(candidates, constraints=constraints)
        best = eval_result.get("best", {}).get("item") if eval_result.get("best") else None
        response_text = ""
        if plan.get("intent") == "generate_meal_plan":
            response_text = f\"Here is a simple meal plan (best choice: {best.get('name') if best else 'N/A'}).\"
        else:
            response_text = f\"I found: {best.get('name') if best else 'No suggestion'}.\"

        # Save to memory
        self.memory.append_history("interactions", {"input": user_input, "response": response_text})

        return {"response": response_text, "plan": plan, "candidates": candidates, "evaluation": eval_result, "logs": self.obs.get_logs()}

def run_agent(user_input: str):
    agent = MainAgent()
    result = agent.handle_message(user_input)
    return result["response"]


Overwriting project/main_agent.py


In [229]:
%%writefile project/app.py
from flask import Flask, request, jsonify
from project.main_agent import run_agent

app = Flask(__name__)

@app.route("/run", methods=["POST"])
def run():
    data = request.json or {}
    user_input = data.get("input", "")
    resp = run_agent(user_input)
    return jsonify({"response": resp})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8080)


Overwriting project/app.py


In [225]:
%%writefile project/run_demo.py
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))
from project.main_agent import run_agent
if __name__ == "__main__":
    print(run_agent("Hello! This is a demo."))


Overwriting project/run_demo.py


In [226]:
%%writefile project/requirements.txt
flask


Overwriting project/requirements.txt


In [227]:
# Test the agent
from project.main_agent import run_agent
print(run_agent("My target calories are 1500!"))

I found: my.


In [228]:
# Zip the project folder
!zip -r project.zip project -q
print("project.zip created")

project.zip created
